<a href="https://colab.research.google.com/github/fooodforrhino/wood_id/blob/main/test/test_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
<a target="_blank" href="https://colab.research.google.com/github/fooodforrhino/wood_id/edit/main/test/test_yolo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>